In [1]:
%run ../config/initialize.ipynb

In [2]:
# %run 01_time_date.ipynb
# %run 02_teams.ipynb
# %run 03_matchup.ipynb
# %run 04_travel.ipynb
# %run 05_homeaway.ipynb
# %run 06_weather.ipynb
# %run 07_line.ipynb

In [14]:
out_tbl_game = 'features.combined_0127'
out_tbl_labels = 'labels.combined_0127'

In [15]:
key = ['game_id','h_team_id','v_team_id']

In [16]:
tables = ['features.time_date',
    'features.home_field',
    'features.travel',
    'features.weather',
    'features.matchup',
    'features.line',
    'features.team_history']

In [17]:
game = spark.table('game').select(*key).cache()
before = game.count()
for t in tables:
    curr = spark.table(t)
    for f in set(curr.columns) - set(['game_id']):
        curr = curr.withColumnRenamed(f, '{}___{}'.format(
                                                t.split('.')[1], f)
                                            )
    game = game.join(curr, on='game_id')
    assert game.count() == before

In [18]:
features = filter(lambda x: x not in key, game.columns)

pd.options.display.max_rows = 1000
game.limit(5).toPandas().T

,0,1,2,3,4
game_id,201511150ram,201511090sdg,201510180det,201512270tam,201510110kan
h_team_id,ram,sdg,det,tam,kan
v_team_id,chi,chi,chi,chi,chi
time_date___game_time_eastern,13,21,13,13,13
time_date___game_time_local,12,18,13,13,12
time_date___day_of_week,6,0,6,6,6
time_date___is_sunday,1,0,1,1,1
time_date___week_id,10,9,6,16,5
time_date___is_playoffs,0,0,0,0,0
time_date___is_superbowl,0,0,0,0,0


### Ensure no nulls

In [8]:
for i, f in enumerate(features):
    tmp = game.withColumn('{}_isnull'.format(f), F.when(col(f).isNull(), 1).otherwise(0))\
        .groupby('{}_isnull'.format(f))\
        .count()\
        .toPandas()
    if tmp[tmp['{}_isnull'.format(f)] != 0].shape[0] > 0:
        f
        tmp

### Write out games and labels tables
* join with one another to ensure they're the same size

In [19]:
out_tbl_game
out_tbl_labels

'features.combined_0127'

'labels.combined_0127'

In [25]:
labels = spark.table('labels.team_game_line_labels')\
            .select('game_id','is_home','did_cover_sbr','did_win')\
            .filter(
                (col('is_home') == 1)
                & (col('did_cover_sbr') != 0)
                & (col('did_win') != 0)
            ).drop('is_home')\
            .join(game.select('game_id'), on='game_id')

In [26]:
game.join(
    labels.select('game_id'), on='game_id'
).write.mode('overwrite').saveAsTable(out_tbl_game)

In [27]:
labels.groupby('did_cover_sbr').count().toPandas()
labels.groupby('did_win').count().toPandas()

,did_cover_sbr,count
0,-1,1419
1,1,1380


,did_win,count
0,-1,1201
1,1,1598


In [28]:
labels.write.mode('overwrite').saveAsTable(out_tbl_labels)

### Handle H vs. V rows -- interaction features